## IMPORTING LIBRARIES

In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import datetime

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import StratifiedKFold,KFold, train_test_split

## IMPORTING DATASET

In [70]:
train_df=pd.read_csv('Train.csv')
train_df=train_df.drop(['Place_ID'],1)

test_df=pd.read_csv('Test.csv')
test_df=test_df.drop(['Place_ID'],1)

# Data Pre-processing

In [71]:
train_df.columns

Index(['Place_ID X Date', 'Date', 'target', 'target_min', 'target_max',
       'target_variance', 'target_count',
       'precipitable_water_entire_atmosphere',
       'relative_humidity_2m_above_ground',
       'specific_humidity_2m_above_ground', 'temperature_2m_above_ground',
       'u_component_of_wind_10m_above_ground',
       'v_component_of_wind_10m_above_ground',
       'L3_NO2_NO2_column_number_density',
       'L3_NO2_NO2_slant_column_number_density',
       'L3_NO2_absorbing_aerosol_index', 'L3_NO2_cloud_fraction',
       'L3_NO2_sensor_altitude', 'L3_NO2_sensor_azimuth_angle',
       'L3_NO2_sensor_zenith_angle', 'L3_NO2_solar_azimuth_angle',
       'L3_NO2_solar_zenith_angle',
       'L3_NO2_stratospheric_NO2_column_number_density',
       'L3_NO2_tropopause_pressure',
       'L3_NO2_tropospheric_NO2_column_number_density',
       'L3_O3_O3_column_number_density', 'L3_O3_O3_effective_temperature',
       'L3_O3_cloud_fraction', 'L3_O3_sensor_azimuth_angle',
       'L3_O3_s

In [72]:
train_df.shape,test_df.shape

((30557, 81), (16136, 76))

In [73]:
train_df.head(3)

,Place_ID X Date,Date,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,...,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,010Q650 X 2020-01-02,1/2/2020,38.0,23.0,53.0,769.50,92,11.0,60.200001,0.00804,...,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118
1,010Q650 X 2020-01-03,1/3/2020,39.0,25.0,63.0,1319.85,91,14.6,48.799999,0.00839,...,59.624912,-67.693509,28.614804,1789.960449,3384.226563,0.015104,75.630043,55.657486,-53.868134,19.293652
2,010Q650 X 2020-01-04,1/4/2020,24.0,8.0,56.0,1181.96,96,16.4,33.400002,0.00750,...,49.839714,-78.342701,34.296977,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
test_df.head(3)

,Place_ID X Date,Date,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,...,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,0OS9LVX X 2020-01-02,1/2/2020,11.600000,30.200001,0.00409,14.656824,3.956377,0.712605,0.000053,0.000108,...,1.445658,-95.984984,22.942019,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0OS9LVX X 2020-01-03,1/3/2020,18.300001,42.900002,0.00595,15.026544,4.230430,0.661892,0.000050,0.000109,...,34.641758,-95.014908,18.539116,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0OS9LVX X 2020-01-04,1/4/2020,17.600000,41.299999,0.00590,15.511041,5.245728,1.640559,0.000050,0.000134,...,55.872276,-94.015418,14.140820,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30557 entries, 0 to 30556
Data columns (total 81 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Place_ID X Date                                      30557 non-null  object 
 1   Date                                                 30557 non-null  object 
 2   target                                               30557 non-null  float64
 3   target_min                                           30557 non-null  float64
 4   target_max                                           30557 non-null  float64
 5   target_variance                                      30557 non-null  float64
 6   target_count                                         30557 non-null  int64  
 7   precipitable_water_entire_atmosphere                 30557 non-null  float64
 8   relative_humidity_2m_above_ground                    30557 non-nul

In [76]:
train_df.isnull().sum().tail(60)

L3_NO2_solar_zenith_angle                               2190
L3_NO2_stratospheric_NO2_column_number_density          2190
L3_NO2_tropopause_pressure                              2190
L3_NO2_tropospheric_NO2_column_number_density           8669
L3_O3_O3_column_number_density                           283
L3_O3_O3_effective_temperature                           283
L3_O3_cloud_fraction                                     283
L3_O3_sensor_azimuth_angle                               283
L3_O3_sensor_zenith_angle                                283
L3_O3_solar_azimuth_angle                                283
L3_O3_solar_zenith_angle                                 283
L3_CO_CO_column_number_density                          5423
L3_CO_H2O_column_number_density                         5423
L3_CO_cloud_height                                      5423
L3_CO_sensor_altitude                                   5423
L3_CO_sensor_azimuth_angle                              5423
L3_CO_sensor_zenith_angl

In [77]:
train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df['Month'] = train_df['Date'].dt.month
train_df['Day'] = train_df['Date'].dt.day
train_df['Dayofweek'] = train_df['Date'].dt.dayofweek
train_df['DayOfyear'] = train_df['Date'].dt.dayofyear
train_df['WeekOfyear'] = train_df['Date'].dt.weekofyear

test_df['Date'] = pd.to_datetime(test_df['Date'])
test_df['Month'] = test_df['Date'].dt.month
test_df['Day'] = test_df['Date'].dt.day
test_df['Dayofweek'] = test_df['Date'].dt.dayofweek
test_df['DayOfyear'] = test_df['Date'].dt.dayofyear
test_df['WeekOfyear'] = test_df['Date'].dt.weekofyear

In [79]:
lag_features = ['relative_humidity_2m_above_ground',
                'v_component_of_wind_10m_above_ground',
                'L3_NO2_NO2_column_number_density',
                'L3_HCHO_tropospheric_HCHO_column_number_density',
                'L3_CLOUD_surface_albedo',
                'L3_CO_CO_column_number_density',
                'temperature_2m_above_ground',
                'L3_CLOUD_cloud_optical_depth',
                'L3_O3_O3_column_number_density'
                ]

for feat in lag_features:
    for i in range(2):
        train_df[feat+'_lag'+str(i+1)] = train_df[feat].shift(i+1)
        test_df[feat+'_lag'+str(i+1)] = test_df[feat].shift(i+1)

In [80]:
NO2_Column_number_density_mean=train_df['L3_NO2_NO2_slant_column_number_density'].mean()
O3_column_number_density_mean=train_df['L3_O3_O3_column_number_density'].mean()
CO_CO_column_number_density_mean=train_df['L3_CO_CO_column_number_density'].mean()
CO_H2O_column_number_density_mean=train_df['L3_CO_H2O_column_number_density'].mean() 
HCHO_slant_column_number_density_mean=train_df['L3_HCHO_HCHO_slant_column_number_density'].mean()
CLOUD_cloud_top_pressure_mean=train_df['L3_CLOUD_cloud_top_pressure'].mean()
AER_AI_absorbing_aerosol_index_mean=train_df['L3_AER_AI_absorbing_aerosol_index'].mean()
CH4_aerosol_optical_depth_mean=train_df['L3_CH4_aerosol_optical_depth'].mean()
SO2_slant_column_number_density_mean=train_df['L3_SO2_SO2_slant_column_number_density'].mean()

NO2_Column_number_density_mean=test_df['L3_NO2_NO2_slant_column_number_density'].mean()
O3_column_number_density_mean=test_df['L3_O3_O3_column_number_density'].mean()
CO_CO_column_number_density_mean=test_df['L3_CO_CO_column_number_density'].mean()
CO_H2O_column_number_density_mean=test_df['L3_CO_H2O_column_number_density'].mean() 
HCHO_slant_column_number_density_mean=test_df['L3_HCHO_HCHO_slant_column_number_density'].mean()
CLOUD_cloud_top_pressure_mean=test_df['L3_CLOUD_cloud_top_pressure'].mean()
AER_AI_absorbing_aerosol_index_mean=test_df['L3_AER_AI_absorbing_aerosol_index'].mean()
CH4_aerosol_optical_depth_mean=test_df['L3_CH4_aerosol_optical_depth'].mean()
SO2_slant_column_number_density_mean=test_df['L3_SO2_SO2_slant_column_number_density'].mean()

## FILLING MISSING VALUES

In [81]:
train_df['L3_NO2_NO2_slant_column_number_density']=train_df['L3_NO2_NO2_slant_column_number_density'].fillna(NO2_Column_number_density_mean)
train_df['L3_O3_O3_column_number_density']=train_df['L3_O3_O3_column_number_density'].fillna(O3_column_number_density_mean)
train_df['L3_CO_CO_column_number_density']=train_df['L3_CO_CO_column_number_density'].fillna(CO_CO_column_number_density_mean)
train_df['L3_CO_H2O_column_number_density']=train_df['L3_CO_H2O_column_number_density'].fillna(CO_H2O_column_number_density_mean)
train_df['L3_HCHO_HCHO_slant_column_number_density']=train_df['L3_HCHO_HCHO_slant_column_number_density'].fillna(HCHO_slant_column_number_density_mean)
train_df['L3_CLOUD_cloud_top_pressure']=train_df['L3_CLOUD_cloud_top_pressure'].fillna(CLOUD_cloud_top_pressure_mean)
train_df['L3_AER_AI_absorbing_aerosol_index']=train_df['L3_AER_AI_absorbing_aerosol_index'].fillna(AER_AI_absorbing_aerosol_index_mean)
train_df['L3_CH4_aerosol_optical_depth']=train_df['L3_CH4_aerosol_optical_depth'].fillna(CH4_aerosol_optical_depth_mean)
train_df['L3_SO2_SO2_slant_column_number_density']=train_df['L3_SO2_SO2_slant_column_number_density'].fillna(SO2_slant_column_number_density_mean)


test_df['L3_NO2_NO2_slant_column_number_density']=test_df['L3_NO2_NO2_slant_column_number_density'].fillna(NO2_Column_number_density_mean)
test_df['L3_O3_O3_column_number_density']=test_df['L3_O3_O3_column_number_density'].fillna(O3_column_number_density_mean)
test_df['L3_CO_CO_column_number_density']=test_df['L3_CO_CO_column_number_density'].fillna(CO_CO_column_number_density_mean)
test_df['L3_CO_H2O_column_number_density']=test_df['L3_CO_H2O_column_number_density'].fillna(CO_H2O_column_number_density_mean)
test_df['L3_HCHO_HCHO_slant_column_number_density']=test_df['L3_HCHO_HCHO_slant_column_number_density'].fillna(HCHO_slant_column_number_density_mean)
test_df['L3_CLOUD_cloud_top_pressure']=test_df['L3_CLOUD_cloud_top_pressure'].fillna(CLOUD_cloud_top_pressure_mean)
test_df['L3_AER_AI_absorbing_aerosol_index']=test_df['L3_AER_AI_absorbing_aerosol_index'].fillna(AER_AI_absorbing_aerosol_index_mean)
test_df['L3_CH4_aerosol_optical_depth']=test_df['L3_CH4_aerosol_optical_depth'].fillna(CH4_aerosol_optical_depth_mean)
test_df['L3_SO2_SO2_slant_column_number_density']=test_df['L3_SO2_SO2_slant_column_number_density'].fillna(SO2_slant_column_number_density_mean)

In [82]:
train_df.fillna(-99,inplace=True)
test_df.fillna(-99,inplace=True)

In [83]:
train_df.isnull().sum().tail(60)

L3_HCHO_solar_azimuth_angle                             0
L3_HCHO_solar_zenith_angle                              0
L3_HCHO_tropospheric_HCHO_column_number_density         0
L3_HCHO_tropospheric_HCHO_column_number_density_amf     0
L3_CLOUD_cloud_base_height                              0
L3_CLOUD_cloud_base_pressure                            0
L3_CLOUD_cloud_fraction                                 0
L3_CLOUD_cloud_optical_depth                            0
L3_CLOUD_cloud_top_height                               0
L3_CLOUD_cloud_top_pressure                             0
L3_CLOUD_sensor_azimuth_angle                           0
L3_CLOUD_sensor_zenith_angle                            0
L3_CLOUD_solar_azimuth_angle                            0
L3_CLOUD_solar_zenith_angle                             0
L3_CLOUD_surface_albedo                                 0
L3_AER_AI_absorbing_aerosol_index                       0
L3_AER_AI_sensor_altitude                               0
L3_AER_AI_sens

In [85]:
train_df.drop(['Place_ID X Date','Date','target_min','target_max','target_variance','target_count',
            'L3_CH4_CH4_column_volume_mixing_ratio_dry_air',
       'L3_CH4_aerosol_height', 'L3_CH4_aerosol_optical_depth',
       'L3_CH4_sensor_azimuth_angle', 'L3_CH4_sensor_zenith_angle',
       'L3_CH4_solar_azimuth_angle', 'L3_CH4_solar_zenith_angle'], axis=1, inplace=True)
test_df.drop(['Place_ID X Date','Date', 'L3_CH4_CH4_column_volume_mixing_ratio_dry_air',
       'L3_CH4_aerosol_height', 'L3_CH4_aerosol_optical_depth',
       'L3_CH4_sensor_azimuth_angle', 'L3_CH4_sensor_zenith_angle',
       'L3_CH4_solar_azimuth_angle', 'L3_CH4_solar_zenith_angle'], axis=1, inplace=True)

# MODEL BUILDING

## Train test split

In [88]:
x=np.array(train_df.drop(['target'],1))
y=np.array(train_df['target'])

In [89]:
len(x)

30557

In [90]:
len(y)

30557

In [93]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=1)

In [94]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((22917, 90), (7640, 90), (22917,), (7640,))

In [97]:
train_data = lgb.Dataset(x_train, label=y_train)
test_data = lgb.Dataset(x_test, label=y_test)

param = {'objective': 'regression',
         'boosting': 'gbdt',  
         'metric': 'rmse',
         'learning_rate': 0.05, 
         'num_iterations': 7500,
         'max_depth': -1,
         'min_data_in_leaf': 40,
         'bagging_fraction': 0.8,
         'bagging_freq': 1,
         'feature_fraction': 0.5
         }

         
clf = lgb.train(params=param, 
                early_stopping_rounds=100,
                verbose_eval=500,
                train_set=train_data,
                valid_sets=[test_data])

y_pred = clf.predict(x_test) 

Training until validation scores don't improve for 100 rounds
[500]	valid_0's rmse: 25.6418
[1000]	valid_0's rmse: 24.9133
[1500]	valid_0's rmse: 24.5872
[2000]	valid_0's rmse: 24.3761
[2500]	valid_0's rmse: 24.2359
[3000]	valid_0's rmse: 24.1609
[3500]	valid_0's rmse: 24.1186
[4000]	valid_0's rmse: 24.0851
Early stopping, best iteration is:
[4282]	valid_0's rmse: 24.0705


In [98]:
np.sqrt(mean_squared_error(y_test, y_pred))

24.070549707067336

## TESTING PREDICTION

In [104]:
errlgb = []
y_pred_totlgb = []

fold = KFold(n_splits=10, shuffle=True, random_state=42)

for train_index, test_index in fold.split(x):
    
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    train_data = lgb.Dataset(x_train, label=y_train)
    test_data = lgb.Dataset(x_test, label=y_test)
    
    clf = lgb.train(params=param, 
                     early_stopping_rounds=200,
                     verbose_eval=500,
                     train_set=train_data,
                     valid_sets=[test_data])

    y_pred = clf.predict(x_test) 

    print("RMSE: ", np.sqrt(mean_squared_error(y_test, y_pred)))
    
    errlgb.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    
    p = clf.predict(test_df)
    
    y_pred_totlgb.append(p)

Training until validation scores don't improve for 200 rounds
[500]	valid_0's rmse: 25.0524
[1000]	valid_0's rmse: 23.9795
[1500]	valid_0's rmse: 23.5383
[2000]	valid_0's rmse: 23.2866
[2500]	valid_0's rmse: 23.1397
[3000]	valid_0's rmse: 23.0364
[3500]	valid_0's rmse: 22.952
[4000]	valid_0's rmse: 22.9054
[4500]	valid_0's rmse: 22.8642
[5000]	valid_0's rmse: 22.8379
[5500]	valid_0's rmse: 22.8051
[6000]	valid_0's rmse: 22.7858
[6500]	valid_0's rmse: 22.7653
[7000]	valid_0's rmse: 22.7525
[7500]	valid_0's rmse: 22.7412
Did not meet early stopping. Best iteration is:
[7497]	valid_0's rmse: 22.7408
RMSE:  22.74076396391936
Training until validation scores don't improve for 200 rounds
[500]	valid_0's rmse: 28.7138
[1000]	valid_0's rmse: 27.9805
[1500]	valid_0's rmse: 27.5817
[2000]	valid_0's rmse: 27.3494
[2500]	valid_0's rmse: 27.2222
[3000]	valid_0's rmse: 27.1275
[3500]	valid_0's rmse: 27.0609
[4000]	valid_0's rmse: 27.0054
[4500]	valid_0's rmse: 26.9554
[5000]	valid_0's rmse: 26.9122


In [105]:
prediction=y_pred_totlgb
prediction

[array([56.98261225, 34.91216573, 37.1369619 , ..., 54.9858163 ,
        35.88709198, 33.90851985]),
 array([44.90021624, 34.71941388, 29.73909075, ..., 58.28843879,
        39.13947611, 30.64930186]),
 array([63.15854221, 24.39026015, 30.57436861, ..., 53.22962207,
        37.82661271, 34.37397189]),
 array([46.44992413, 38.40833904, 41.11055253, ..., 54.04096003,
        41.56883894, 34.34290668]),
 array([51.09060599, 25.06799382, 38.21619499, ..., 53.3697738 ,
        42.48712767, 28.71677776]),
 array([53.51905876, 25.08528838, 49.96267182, ..., 56.28213134,
        38.57565511, 38.42532625]),
 array([56.45890579, 28.04897712, 36.31830051, ..., 54.46232783,
        40.26088904, 28.15836438]),
 array([85.91801199, 36.32726237, 49.23574227, ..., 50.76710274,
        45.07290248, 27.28111451]),
 array([62.05492436, 26.72841965, 25.31007689, ..., 55.44181408,
        42.0022243 , 32.80407747]),
 array([78.13286963, 21.37808729, 30.06700784, ..., 48.30846417,
        36.57790856, 26.02

In [108]:
preds=np.mean(y_pred_totlgb,0)
preds

array([59.86656714, 29.50662074, 36.76709681, ..., 53.91764511,
       39.93987269, 31.46891605])

In [109]:
Submission=pd.read_csv('Submission.csv')

In [110]:
Submission.head()

,Unnamed: 0,Place_ID X Date,target
0,0,0OS9LVX X 2020-01-02,38
1,1,0OS9LVX X 2020-01-03,30
2,2,0OS9LVX X 2020-01-04,35
3,3,0OS9LVX X 2020-01-05,33
4,4,0OS9LVX X 2020-01-06,32


In [112]:
Submission['target']=preds

In [113]:
Submission.to_csv('Submission11.csv',index=False)

In [114]:
np.mean(errlgb,0)

24.4449304619692